In [1]:
# magics to auto-reload external modules in case you are making changes to langchain while working on this notebook
%load_ext autoreload
%autoreload 2

In [1]:
# get a token: https://replicate.com/account

from getpass import getpass

REPLICATE_API_TOKEN = getpass()

In [2]:
import os

os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [7]:
%pip install -q langchain
%pip install -q langchain-community
%pip install -q langchain-core
%pip install -q replicate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chains import LLMChain
from langchain_community.llms import Replicate
from langchain_core.prompts import PromptTemplate

In [4]:
input = {
    "prompt": "Send a email from andreas86@telia.se to myfriend@telia.se where you discuss the weather. In the body, describe the current weather in Stockholm as detailed as possible.\n\nRespond with json that adheres to the following jsonschema:\n\n{jsonschema}",
    "grammar": "",
    "jsonschema": "{\n \"type\": \"object\", \n \"properties\": {\n \"Id\" : {  \"type\" : \"integer\"}, \n \"name\" : {\n \"type\" : \"string\"}, \n \"age\" : {\n \"type\" : \"integer\"}, \n \"gender\" : {\n \"type\" : \"string\" }, \n \"diagnosis\" : {\n \"type\" : \"string\" }} }"
}

llm = Replicate(
    model="andreasjansson/llama-2-13b-chat-gguf:60ec5dda9ff9ee0b6f786c9d1157842e6ab3cc931139ad98fe99e08a35c5d4d4",
    model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1, "jsonschema": "{\n \"type\": \"object\", \n \"properties\": {\n \"Id\" : {  \"type\" : \"integer\"}, \n \"name\" : {\n \"type\" : \"string\"}, \n \"age\" : {\n \"type\" : \"integer\"}, \n \"gender\" : {\n \"type\" : \"string\" }, \n \"diagnosis\" : {\n \"type\" : \"string\" }} }"},
)
prompt = """
User: A male patient Alex is 20 years old have headache. Tell me what is the patient name, age, gender, and diagnosis
"""
llm(prompt)

/home/laansdole/langchain-replicate/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'{ "Id": 1, "age": 20, "diagnosis": "headache", "gender": "male" ,"name": "Alex" } '

In [5]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Replicate(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="andreasjansson/llama-2-13b-chat-gguf:60ec5dda9ff9ee0b6f786c9d1157842e6ab3cc931139ad98fe99e08a35c5d4d4",
    model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1, "jsonschema": "{\n \"type\": \"object\", \n \"properties\": {\n \"Id\" : {  \"type\" : \"integer\"}, \n \"name\" : {\n \"type\" : \"string\"}, \n \"age\" : {\n \"type\" : \"integer\"}, \n \"gender\" : {\n \"type\" : \"string\" }, \n \"diagnosis\" : {\n \"type\" : \"string\" }} }"},
)
prompt = """
User: A male patient Alex is 20 years old have headache. Tell me what is the patient name, age, gender, and diagnosis
Assistant:
"""
_ = llm.invoke(prompt)

{ "Id": 1, "age": 20, "diagnosis": "tension headache", "gender": "male" ,"name": "Alex" }